#### Importing libraries

In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd

#### Extracting Raw HTML

In [2]:
url_temp = 'https://www.indeed.com/jobs?q={}&l={}&radius=100' #Data%20Scientist, New%20York

In [3]:
def get_url(position, location):
    template = url_temp.format(position, location)
    url = template.format(position, location) 
    return url

In [4]:
get_url('Accountant', "San Francisco")

'https://www.indeed.com/jobs?q=Accountant&l=San Francisco&radius=100'

#  

##### Plugging in JOB POSITION and LOCATION 

In [5]:
get_url('Data Analyst', 'New York')

'https://www.indeed.com/jobs?q=Data Analyst&l=New York&radius=100'

##### To check if the connection is there 

In [6]:
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=New%20York%2C%20NY&vjk=1f961d8306d040e1'

In [7]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

response = requests.get(url, headers = headers) 
if response.status_code == requests.codes.ok:
    print('Yes, connection is there')

Yes, connection is there


In [8]:
url

'https://www.indeed.com/jobs?q=data%20analyst&l=New%20York%2C%20NY&vjk=1f961d8306d040e1'

In [123]:
data = {
    'job_title' : [],
    'comp_name' : [],
    'location' : [],
    'salary' : [],
    #'skills' : [],
    #'work_exp' : [],
    'post_url' : []
}


#  

#### Getting everything from a single page 

In [117]:
#def extract(position, location): 
    #headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    #url = f'https://www.indeed.com/jobs?q={position}&l={location}&radius=50'
    #url = template.format(position, location)
    #url = f'https://www.indeed.com/jobs?q=data%20analyst&l=New%20York%2C%20NY&start={page}&vjk=1f961d8306d040e1'
    #r = requests.get(url, headers)
    #if r.status_code == requests.codes.ok
    #print('Yes, connection is there')
    #soup = BeautifulSoup(r.content, 'html.parser')
    
    #for page in soup.find_all('tbody'): #outter most tbody(alsmost the whole page) and going down 
        #trs = page.find('tr')
        #for i in trs.find_all('a', href = True): # gets all <a></a>'s
            #if i.find('a', href = True): # gets <a></a>'s for each job posts
                #job_url = 'https://www.indeed.com' + i['href']
                #print(job_url)
                #data['post_url'].append(job_url)   
    #return soup

def extract(page): 
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    url = f'https://www.indeed.com/jobs?q=data%20analyst&l=New%20York%2C%20NY&start={page}&vjk=1f961d8306d040e1'
    r = requests.get(url, headers)
    #if r.status_code == requests.codes.ok
    #print('Yes, connection is there')
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

def transform(soup):
    divs = soup.find_all('div', {'class' : 'job_seen_beacon'}) #slider_container also works
    #print(divs)
    
    """"Getting a job title"""
    for k in divs:
        #title = k.find('span').text #eto kogda [0] - has 'new' items 
        #title = k.find('div', 'jobTitle jobTitle-color-purple').text #[1]
        #if k.find('span').text == 'new':
        title = k.find('h2', 'jobTitle', 'title').text.strip('new')
        
        #print(title)
        data['job_title'].append(title)
   
    """"Getting a company name"""
    for l in divs:
        #company_name = l.find('span', {'class':'companyName'}).text
        company_name = l.find('span', 'companyName').text
        #print(company_name)
        data['comp_name'].append(company_name)
                
    """"Getting a location"""
    for m in divs:
        location = m.find('div', {'class': 'companyLocation'}).text
        #print(location)
        data['location'].append(location)
        
    """"Getting a salary if it is available"""
    for n in divs:
        if n.find_all('div', {'class': 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}):
            for o in n.find_all('div', {'class': 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}):
                salary = o.text
                data['salary'].append(salary)
        else:
            salary = 'None'
            data['salary'].append(salary)
        #print(salary)
        
    """"Getting job URLs"""  
    for page in soup.find_all('tbody'): #outter most tbody(alsmost the whole page) and going down 
        trs = page.find('tr')
        for i in trs.find_all('a', href = True): # gets all <a></a>'s
            if i.find('a', href = True): # gets <a></a>'s for each job posts
                job_url = 'https://www.indeed.com' + i['href']
                #print(job_url)
                data['post_url'].append(job_url)
                

In [124]:
c = extract(10)
transform(c)

In [125]:
data

{'job_title': ['Data Quality Analyst',
  'Business Analyst',
  'Data Analyst',
  'Data and Visualization Specialist',
  'Data Analyst',
  'Junior Analyst',
  'Junior Data Analyst',
  'Data Analyst',
  'Data & Policy Analyst',
  "Data Analyst (Legal support) Supporting the US Attorney's Of...",
  'Exemptions Data Analyst',
  'Data Analyst',
  'Quality Assurance Analyst 100% Remot',
  'Entry Level Business Analyst',
  'Data Analyst'],
 'comp_name': ['NYC Careers',
  'The Mom Project',
  'Brooksource',
  'Talenthub Worldwide, Inc.',
  'Network Rail Consulting',
  'BNY Mellon',
  'Evalueserve',
  'Fidelis Care',
  'NYC DEPARTMENT OF FINANCE',
  'FSA Federal',
  'NYC Careers',
  'NYC Criminal Justice Agency',
  'Actalent',
  'CloudServiceTek',
  'Columbia University'],
 'location': ['Brooklyn, NY 11201',
  'New York, NY',
  'New York, NY•Temporarily Remote',
  'New York, NY•Remote',
  'New York, NY 10018 (Clinton area)',
  'New York, NY',
  'New York, NY•Temporarily Remote',
  'Long Island 

In [126]:
print(len(data['job_title']))
print(len(data['comp_name']))
print(len(data['location']))
print(len(data['salary']))
print(len(data['post_url']))

15
15
15
15
13


#### Going through pages using range() function

In [146]:
for i in range(0,200,10):
    c = extract('data scientist', 'san francisco')
    transform(c)
#print(len(data['job_title']))
#print(len(data['comp_name']))
#print(len(data['location']))
#print(len(data['salary']))
#print(len(data['post_url']))


In [148]:
print(len(data['job_title']))
print(len(data['comp_name']))
print(len(data['location']))
print(len(data['salary']))
print(len(data['post_url']))

315
315
315
315
255


#  

#### Getting next page by finding paginator 

In [ ]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=New%20York%2C%20NY&start=0&vjk=1f961d8306d040e1'
r = requests.get(url, headers)
    #if r.status_code == requests.codes.ok
    #print('Yes, connection is there')
soup = BeautifulSoup(r.content, 'html.parser')
soup.find('a', {'aria-label' : 'Next'}).get('href') #this gets us the URL of the next page





In [180]:
data = {
    'job_title' : [],
    'comp_name' : [],
    'location' : [],
    'salary' : [],
    #'skills' : [],
    #'work_exp' : [],
    'post_url' : []
}

#### Putting everything together with page paginator  

In [179]:
data = {
    'job_title' : [],
    'comp_name' : [],
    'location' : [],
    'salary' : [],
    #'skills' : [],
    #'work_exp' : [],
    'post_url' : []
}


def extract(position, location): 
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    template = 'https://www.indeed.com/jobs?q={}&l={}&radius=50'
    url = template.format(position, location)
    #url = f'https://www.indeed.com/jobs?q=data%20analyst&l=New%20York%2C%20NY&start={page}&vjk=1f961d8306d040e1'
    r = requests.get(url, headers)
    #if r.status_code == requests.codes.ok
    #print('Yes, connection is there')
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

def transform(soup):
    divs = soup.find_all('div', {'class' : 'job_seen_beacon'}) #slider_container also works
    #print(divs)
    
    """"Getting a job title"""
    for k in divs:
        #title = k.find('span').text #eto kogda [0] - has 'new' items 
        #title = k.find('div', 'jobTitle jobTitle-color-purple').text #[1]
        #if k.find('span').text == 'new':
        title = k.find('h2', 'jobTitle', 'title').text.strip('new')
        
        #print(title)
        data['job_title'].append(title)
   
    """"Getting a company name"""
    for l in divs:
        #company_name = l.find('span', {'class':'companyName'}).text
        company_name = l.find('span', 'companyName').text
        #print(company_name)
        data['comp_name'].append(company_name)
                
    """"Getting a location"""
    for m in divs:
        location = m.find('div', {'class': 'companyLocation'}).text
        #print(location)
        data['location'].append(location)
        
    """"Getting a salary if it is available"""
    for n in divs:
        if n.find_all('div', {'class': 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}):
            for o in n.find_all('div', {'class': 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}):
                salary = o.text
                data['salary'].append(salary)
        else:
            salary = 'None'
            data['salary'].append(salary)
        #print(salary)
        
    """"Getting job URLs"""  
    for page in soup.find_all('tbody'): #outter most tbody(alsmost the whole page) and going down 
        trs = page.find('tr')
        for i in trs.find_all('a', href = True): # gets all <a></a>'s
            if i.find('a', href = True): # gets <a></a>'s for each job posts
                job_url = 'https://www.indeed.com' + i['href']
                #print(job_url)
                data['post_url'].append(job_url)


In [183]:
c = extract('Data Scientist', 'Chicago')
transform(c)

https://www.indeed.com/company/SD-Mayer-and-Associates/jobs/Staff-Accountant-03a1d9298dec59d3?fccid=077e6eb877b48fc1&vjs=3
https://www.indeed.com/rc/clk?jk=03471806201086ce&fccid=de115aaf8dbb353e&vjs=3
https://www.indeed.com/company/FriendFinder-Networks/jobs/Junior-Accountant-dfd6abc45f8d0b92?fccid=702c7d72f9a9a730&vjs=3
https://www.indeed.com/rc/clk?jk=cd31a68fef986bc7&fccid=807f5e2015618bb2&vjs=3
https://www.indeed.com/rc/clk?jk=efc30142e7e2c090&fccid=2a341562d64c7cdb&vjs=3
https://www.indeed.com/company/Page-&-Turnbull/jobs/Part-Time-Accountant-4bbbd778e065b90a?fccid=dff50c6a548bec90&vjs=3
https://www.indeed.com/company/Juniper-Square/jobs/Associate-Fund-Accountant-d47e4a78ca5e3a91?fccid=1b53a4c49b4135ed&vjs=3
https://www.indeed.com/rc/clk?jk=572e35161071f53f&fccid=4027cfd917e1ee29&vjs=3
https://www.indeed.com/company/Zumper/jobs/Account-Payable-Accountant-9a505eafc9d0ccfd?fccid=68b09cbcad949ffc&vjs=3
https://www.indeed.com/rc/clk?jk=b98ea18b44f479ef&fccid=d47acb5b5596d9e4&vjs=3
ht

In [154]:
data_df = pd.DataFrame(data)

In [155]:
data_df

,job_title,comp_name,location,salary,post_url
0,Senior Data Scientist - Ranking Team,Indeed,"Chicago, IL•Temporarily Remote","$123,000 - $179,000 a year",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,Nielsen,"Chicago, IL•Remote",None,https://www.indeed.com/rc/clk?jk=9b8e8f066872d...
2,Data Scientist,Oak Street Health,"Chicago, IL 60603 (The Loop area)•Remote",None,https://www.indeed.com/rc/clk?jk=d66dd8400f1e9...
3,Data Scientist,"Blue Cross Blue Shield of IL, MT, NM, OK & TX","Chicago, IL+5 locations",None,https://www.indeed.com/rc/clk?jk=058f056bddd78...
4,Sr. Data Scientist-Direct Hire (No Subcontractor),Modernagile Technologies,"Chicago, IL","$110,000 - $120,000 a year",https://www.indeed.com/company/Modernagile-Tec...
5,Data Scientist,"Eligo Energy, LLC","Chicago, IL",None,"https://www.indeed.com/company/Eligo-Energy,-L..."
6,Junior Data Scientist,Abbott Laboratories,"Waukegan, IL",None,https://www.indeed.com/rc/clk?jk=19d7778532e4b...
7,Data Scientist,Cognizant Technology Solutions,"Chicago, IL 60601 (Loop area)",None,https://www.indeed.com/rc/clk?jk=263a7ae4d5745...
8,Data Scientist,Cook County Clerk's Office,"Chicago, IL 60602 (The Loop area)","$62,913 - $79,930 a year",https://www.indeed.com/rc/clk?jk=397a1c5a3b8c6...
9,Associate Data Scientist,WALGREENS,"Chicago, IL 60607 (West Loop Gate area)+1 loca...",None,https://www.indeed.com/rc/clk?jk=b8b108b4fdd18...
